In [ ]:
# Luis Enrique Camaños Rebollo      


import pandas as pd
from pandas import Series,DataFrame
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt
import math
from sklearn.neural_network import MLPClassifier
import itertools as iter
from sklearn.metrics import accuracy_score,recall_score
import statistics

In [ ]:
##Separa los datos por clase
def define_class_dataset(name_class, class_list,setInicial):
    class_datasets = {}
    for item, clase in enumerate(class_list):
        variable_name = f"class_{item + 1}"
        class_datasets[variable_name] = setInicial.loc[setInicial[name_class] == clase].copy()
    return class_datasets

In [ ]:
##Proceso aleatorio
def randomize_dataset(dataset):
    return dataset.sample(frac=1).reset_index(drop=True)

In [ ]:
##Proceso de estratificación
def stratification_dataset(datasets_dictionary, percentage):
    train_sets = {}
    test_sets = {}
    for variable_name, dataset in datasets_dictionary.items():
        tope = math.floor(len(dataset) * percentage)
        train_sets[variable_name] = dataset.iloc[0:tope]
        test_sets[variable_name] = dataset.iloc[tope:len(dataset)]
    return train_sets, test_sets

In [ ]:
#Definición de hiperparámetros
def hiper_parameters_definition():
    neuronas = [8,12,15,17]
    capas_ocultas = [1,3,5]
    learning_rate_i = [0.12,0.1,0.05]
    momentum = [0.12,0.1,0.05]
    combinaciones = list(iter.product(neuronas,capas_ocultas,learning_rate_i,momentum))
    return neuronas,capas_ocultas,learning_rate_i,momentum,combinaciones

In [ ]:
#Definición de tamaño de pliegues
def num_values_def(train_sets,NumPliegues):
    num_values = {}
    cont = {}
    for variable_name, dataset in train_sets.items():
        num_values[variable_name] = int(len(dataset)/NumPliegues)
        cont[variable_name] = 0
    return num_values,cont

In [ ]:
def test_cross_validation(train_sets,topes_dic,cont):
    test_A_dic = {}
    cont01 = 0
    for variable_name,dataset in train_sets.items():
        topes_values_list = list(topes_dic.values())
        cont_values_list = list(cont.values())
        test_A_dic[variable_name] = dataset.iloc[cont_values_list[cont01]:topes_values_list[cont01]]
        cont01+=1
    Test_A = pd.concat(list(test_A_dic.values())).copy()
    return Test_A

In [ ]:
def train_cross_validation(train_sets,topes_dic,cont):
    cont02 = 0
    train_A_dic = {}
    for variable_name,dataset in train_sets.items():
        topes_values_list = list(topes_dic.values())
        cont_values_list = list(cont.values())
        train_A_dic[variable_name] = pd.concat([dataset.iloc[0:cont_values_list[cont02]],
                                                    dataset.iloc[topes_values_list[cont02]:len(dataset)]],
                                                    axis=0)
    Train_A = pd.concat(list(train_A_dic.values())).copy()
    return Train_A

In [ ]:
def model_definition(combinacion,epoca):
    lista_neurona = []
    for i in range(combinacion[1]):
        lista_neurona.append(combinacion[0])
    clf = MLPClassifier(
                  hidden_layer_sizes= lista_neurona,
                  solver='adam',
                  learning_rate='adaptive',
                  learning_rate_init=combinacion[2],
                  max_iter=int(epoca),
                  momentum=combinacion[3],
                  random_state = 1,
              )
    return clf

In [ ]:
def desviacion_estandar_calc(lista_valores):
    desviacion_estandar = statistics.stdev(lista_valores)
    promedio = statistics.mean(lista_valores)
    return desviacion_estandar,promedio

In [ ]:
def construir_lista_final(listas):
    parametros_finales = []
    for i in range(len(listas[0])):  
        valores = [lista[i] for lista in listas]
        if all(val == valores[0] for val in valores):
            parametros_finales.append(valores[0])
        else:
            min_valor = min(valores)
            if min_valor >= 1:
                min_valor = int(min_valor)
            max_valor = max(valores)
            if max_valor >= 1:
                max_valor = int(max_valor)
            valor_medio = round(sum(valores) / len(valores), 2)
            if valor_medio >= 1:
                valor_medio = int(valor_medio)
            parametros_finales.append([min_valor, max_valor, valor_medio])
    return parametros_finales

In [ ]:
def combinaciones_final_def(lista):
    for i in range(len(lista)):
        if not isinstance(lista[i], list):
            lista[i] = [lista[i]]
    neuronas = lista[0]
    capas_ocultas = lista[1]
    learning_rate_i = lista[2]
    momentum = lista[3]
    combinaciones_finales = list(iter.product(neuronas,capas_ocultas,learning_rate_i,momentum))
    return combinaciones_finales

In [ ]:
if __name__ == "__main__":
    #Lectura de base de datos tratada, extracción de atributos y clases, definición de pocentaje 
    #Train y Test
    setInicial = pd.read_csv('FINALdbCANCER.csv')
    atributosName = setInicial.columns[:-1]
    atributosClase = setInicial.columns[-1]
    clasesName = setInicial[setInicial.columns[-1]].drop_duplicates()
    clasesList = setInicial[atributosClase].unique().tolist()
    percentage = 0.75
    #Separación de datos por clase para futura estratificación
    class_datasets = define_class_dataset(atributosClase, clasesList,setInicial)
    #Proceso aleatorio a los datos de cada clase
    for variable_name, dataset in class_datasets.items():
        class_datasets[variable_name] = randomize_dataset(dataset)
    #Proceso de estratificación que genera los sets Train y Test por clase
    train_sets, test_sets = stratification_dataset(class_datasets, percentage)
    #Generación de conjunto Test y aplicación de proceso aleatorio para eliminar sesgos
    TestG = pd.concat(list(test_sets.values())).copy()
    TestG = randomize_dataset(TestG)
    #Generación de conjunto Train y aplicación de proceso aleatorio para eliminar sesgos
    TrainG = pd.concat(list(train_sets.values())).copy()
    TrainG = randomize_dataset(TrainG)
    #Definición de número de pliegues para cross-validation (K) 
    NumPliegues=3
    num_values,cont = num_values_def(train_sets,NumPliegues)
    #Ciclo principal para cross-validation
    accuracy_list = []
    combinacion_list = []
    recall_list = []
    for x in range(1,NumPliegues+1,1):
        print("Pliegue ",x)
        topes_dic = {}
        #Definición de tamaño de particiones con base en los pliegues
        if x==NumPliegues:
            for variable_name,dataset in train_sets.items():
                topes_dic[variable_name] = len(dataset)
        else:
            for key in sorted(set(cont.keys()) & set(num_values.keys())):
                result = cont[key] + num_values[key]
                topes_dic[key] = result
        #Obtención de conjunto Test de validación cruzada y aplicación de proceso aleatorio
        Test_A = test_cross_validation(train_sets,topes_dic,cont)
        Test_A = randomize_dataset(Test_A)
        #Obtención de conjunto Train de validación cruzada y aplicación de proceso aleatorio
        Train_A = train_cross_validation(train_sets,topes_dic,cont)
        Train_A = randomize_dataset(Train_A)
        #Corrimiento de posición para definir nuevo Test y Train para el siguiente pliegue
        print("Train: ",len(Train_A)," intancias.\nTest: ",len(Test_A)," instancias.")
        for key in sorted(set(cont.keys()) & set(num_values.keys())):
            result = cont[key] + num_values[key]
            cont[key] = result
        #Asociación de etiquetas con los datos
        valoresTrain=Train_A[atributosName]
        valoresTest=Test_A[atributosName]
        clasesTrain=Train_A[atributosClase]
        clasesTest=Test_A[atributosClase]
        #ODefinición de hiperparámetros y sus combinaciones 
        neuronas,capas_ocultas,learning_rate_i,momentum,combinaciones = hiper_parameters_definition()
        epoca_inicial = 1
        epoca_final = 250
        paso = 10
        #Ciclo principal de búsqueda de la rejilla
        #Iteración por combinaciones
        #Definición de parámetros iniciales para obtener el mejor modelo
        epoca_ideal = 0
        accuracy_test_ideal = 0
        accuracy_train_ideal = 0
        combinacion_ideal = []
        training_error_ideal = []
        test_error_ideal = []
        epoca_list_ideal = []
        recall_ideal = 0
        error_min = 1
        for combinacion in combinaciones:
            training_error = []
            test_error = []
            epoca_list = [] 
            #Ciclo para probar experimentos por diferentes épocas
            for epoca in range(epoca_inicial,epoca_final,paso):
                #Definición del modelo
                clf_regular = model_definition(combinacion,epoca)
                #Entrenamiento del modelo
                clf_regular.fit(valoresTrain, clasesTrain)
                #Obtención de etiquetas predichas con el conjunto Test
                predicted_values = clf_regular.predict(valoresTest)
                #Obtención de accuracy y error en el conjunto Test
                accuracy_test = accuracy_score(clasesTest,predicted_values)
                error_test = 1 - accuracy_test
                #Obtención de accuracy y error en el conjunto Train
                accuracy_train = clf_regular.score(valoresTrain,clasesTrain)
                error_train = 1 - accuracy_train
                #Recall
                recall_test = recall_score(clasesTest,predicted_values,pos_label='M',average='binary')
                #Registro de errores y épocas
                training_error.append(error_train)
                test_error.append(error_test)
                epoca_list.append(epoca)
                #Obtención de los parámetros ideales en función del error mínimo
                if error_test < error_min:
                    error_min = error_test
                    combinacion_ideal = combinacion
                    epoca_ideal = epoca
                    accuracy_test_ideal = accuracy_test
                    accuracy_train_ideal = accuracy_train
                    training_error_ideal = training_error
                    test_error_ideal = test_error
                    epoca_list_ideal = epoca_list
                    recall_ideal = recall_test
        #Registro de accuracy
        recall_list.append(recall_ideal)
        accuracy_list.append(accuracy_test_ideal)
        combinacion_list.append(combinacion_ideal)
        #Impresión de resultados
        print("Train accuracy: ", accuracy_train_ideal)
        print("Test accuracy: ", accuracy_test_ideal)
        print("Recall: ",recall_ideal)
        print("Best hiperparameters")
        print("Hidden layers: ",combinacion_ideal[1])
        print("Neurons: ",combinacion_ideal[0])
        print("Learnig Rate: ",combinacion_ideal[2])
        print("Momentum: ",combinacion_ideal[3])
        print("Épocas: ",epoca_ideal)
        plt.plot(epoca_list_ideal, training_error_ideal, label="training error")
        plt.plot(epoca_list_ideal, test_error_ideal, label="test error")
        plt.ylabel("Error")
        plt.xlabel("Épocas")
        plt.ylim(0, 1.0)
        plt.legend()
        plt.show()

    valoresTrain_=TrainG[atributosName]
    valoresTest_=TestG[atributosName]
    clasesTrain_=TrainG[atributosClase]
    clasesTest_=TestG[atributosClase]
    print(recall_list)
    standar_dev,promedio = desviacion_estandar_calc(recall_list)

    print("La desviación estandar fue de: ",standar_dev)
    print("EL promedio fue de: ",promedio)
    print("Validación Final")
    print("Train: ",len(TrainG)," intancias.\nTest: ",len(TestG)," instancias.")
    parametros_finales = construir_lista_final(combinacion_list)
    combinaciones_finales_ = combinaciones_final_def(parametros_finales)
    epoca_ideal_ = 0
    accuracy_test_ideal_ = 0
    accuracy_train_ideal_ = 0
    combinacion_ideal_ = []
    training_error_ideal_ = []
    test_error_ideal_ = []
    epoca_list_ideal_ = []
    error_min_ = 1
    recall_ideal_ = 0
    for combinacion in combinaciones_finales_:
        training_error_ = []
        test_error_ = []
        epoca_list_ = []
        for epoca in range(epoca_inicial,epoca_final,paso):
                #Definición del modelo
                clf_regular_ = model_definition(combinacion,epoca)
                #Entrenamiento del modelo
                clf_regular_.fit(valoresTrain_, clasesTrain_)
                #Obtención de etiquetas predichas con el conjunto Test
                predicted_values_ = clf_regular_.predict(valoresTest_)
                #Obtención de accuracy y error en el conjunto Test
                accuracy_test_ = accuracy_score(clasesTest_,predicted_values_)
                error_test_ = 1 - accuracy_test_
                #Obtención de accuracy y error en el conjunto Train
                accuracy_train_ = clf_regular_.score(valoresTrain_,clasesTrain_)
                error_train_ = 1 - accuracy_train_
                #Recall
                recall_test_ = recall_score(clasesTest_,predicted_values_,pos_label='M',average='binary')
                #Registro de errores y épocas
                training_error_.append(error_train_)
                test_error_.append(error_test_)
                epoca_list_.append(epoca) 
                #Obtención de los parámetros ideales en función del error mínimo
                if error_test_ < error_min_:
                    error_min_ = error_test_
                    combinacion_ideal_ = combinacion
                    epoca_ideal_ = epoca
                    accuracy_test_ideal_ = accuracy_test_
                    accuracy_train_ideal_ = accuracy_train_
                    training_error_ideal_ = training_error_
                    test_error_ideal_ = test_error_
                    epoca_list_ideal_ = epoca_list_
                    recall_ideal_ = recall_test_
    print("Train accuracy: ", accuracy_train_ideal_)
    print("Test accuracy: ", accuracy_test_ideal_)
    print("Recall: ",recall_ideal_)
    print("Best hiperparameters")
    print("Hidden layers: ",combinacion_ideal_[1])
    print("Neurons: ",combinacion_ideal_[0])
    print("Learnig Rate: ",combinacion_ideal_[2])
    print("Momentum: ",combinacion_ideal_[3])
    print("Épocas: ",epoca_ideal_)
    plt.plot(epoca_list_ideal_, training_error_ideal_, label="training error")
    plt.plot(epoca_list_ideal_, test_error_ideal_, label="test error")
    plt.ylabel("Error")
    plt.xlabel("Épocas")
    plt.ylim(0, 1.0)
    plt.legend()
    plt.show()

    